<a href="https://www.kaggle.com/code/jeevikasharma2003/grad2masters-admission-prediction-model?scriptVersionId=236215371" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/graduates-admission-prediction/admission_data.csv
/kaggle/input/students-performance/student_performance_tamilnadu.xlsx
/kaggle/input/students-performance/Student Performance records.csv
/kaggle/input/students-performance/student_performance_tamilnadu.csv
/kaggle/input/graduate-admissions-dataset/Admission_Predict.csv
/kaggle/input/graduateadmissions/regression.R
/kaggle/input/dataset2csv/Readme.md
/kaggle/input/dataset2csv/Admission_Predict_Ver1.1.csv
/kaggle/input/graduate-admission-dataset/Admission_Predict_Ver1.1.csv
/kaggle/input/admission-prediction-data/Admission_Predict.csv
/kaggle/input/student-admission-data/StudentData.csv
/kaggle/input/binary/binary.csv
/kaggle/input/graduate-admission-predictions/Admission_Predict.csv
/kaggle/input/graduate-admission-data/Admission_Predict.csv
/kaggle/input/d/saurabhkamble3333/graduateadmissions/Admission_Predict.csv
/kaggle/input/graduate-admissions/Admission_Predict.csv
/kaggle/input/graduate-admissions/Admission_Predict_Ve

In [2]:
data=pd.read_csv("/kaggle/input/graduate-admissions/Admission_Predict_Ver1.1.csv")
data.head(10)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
5,6,330,115,5,4.5,3.0,9.34,1,0.90
6,7,321,109,3,3.0,4.0,8.20,1,0.75
7,8,308,101,2,3.0,4.0,7.90,0,0.68
8,9,302,102,1,2.0,1.5,8.00,0,0.50
9,10,323,108,3,3.5,3.0,8.60,0,0.45


In [3]:
data.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [4]:
data['University Rating'].value_counts()

University Rating
3    162
2    126
4    105
5     73
1     34
Name: count, dtype: int64

In [5]:
data['Research'].value_counts()

Research
1    280
0    220
Name: count, dtype: int64

In [6]:
data.drop(columns=['Serial No.'],inplace=True)

In [7]:
data.duplicated().sum()

0

In [8]:
#Rename columns for clarity
data.columns = ['GRE', 'TOEFL', 'University_Rating', 'SOP', 'LOR', 'CGPA', 'Research', 'Chance_of_Admit']

In [9]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE                500 non-null    int64  
 1   TOEFL              500 non-null    int64  
 2   University_Rating  500 non-null    int64  
 3   SOP                500 non-null    float64
 4   LOR                500 non-null    float64
 5   CGPA               500 non-null    float64
 6   Research           500 non-null    int64  
 7   Chance_of_Admit    500 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 31.4 KB
None


In [10]:
print(data.describe())

              GRE       TOEFL  University_Rating         SOP        LOR  \
count  500.000000  500.000000         500.000000  500.000000  500.00000   
mean   316.472000  107.192000           3.114000    3.374000    3.48400   
std     11.295148    6.081868           1.143512    0.991004    0.92545   
min    290.000000   92.000000           1.000000    1.000000    1.00000   
25%    308.000000  103.000000           2.000000    2.500000    3.00000   
50%    317.000000  107.000000           3.000000    3.500000    3.50000   
75%    325.000000  112.000000           4.000000    4.000000    4.00000   
max    340.000000  120.000000           5.000000    5.000000    5.00000   

             CGPA    Research  Chance_of_Admit  
count  500.000000  500.000000        500.00000  
mean     8.576440    0.560000          0.72174  
std      0.604813    0.496884          0.14114  
min      6.800000    0.000000          0.34000  
25%      8.127500    0.000000          0.63000  
50%      8.560000    1.000000 

In [11]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.stats import gaussian_kde

# 📊 1. Histogram of GRE Scores

In [12]:
fig=px.histogram(data,x='GRE',nbins=20, title='Distribution of GRE Scores')
fig.update_layout(bargap=0.1,template='plotly_dark', yaxis_title='Number of Students')
fig.show(renderer='iframe')

### 📌 **Inference**: Most applicants have GRE scores between 320-324, indicating a competitive applicant pool.

# 📈 2. KDE Plot of CGPA

In [13]:
data['CGPA'].skew()

-0.02661251731835933

In [14]:
fig = px.histogram(data, x='CGPA', nbins=20, marginal="violin", histnorm='probability density', title='CGPA Distribution')


# Extract the CGPA column
x = data['CGPA']
# Create KDE line using scipy
kde = gaussian_kde(x)
x_range = np.linspace(x.min(), x.max(), 100)
kde_values = kde(x_range)

# Add KDE to the same figure
fig.add_trace(go.Scatter(x=x_range, y=kde_values, mode='lines', name='KDE'))


fig.update_layout(bargap=0.1,template='plotly_dark', yaxis_title='Number of Students')
fig.show(renderer='iframe')


### 📌 **Inference:** CGPA is very slightly left skewed i.e, almost symmetrical with most applicants scoring above 8.5.

# 📊 3. Box Plot for TOEFL

In [15]:
fig = px.box(data, y="TOEFL", title="Box Plot for TOEFL Scores")

fig.update_layout(template='plotly_dark')
fig.show(renderer='iframe')



### 📌 **Inference:** TOEFL scores are fairly consistent among applicants.

# 📊 4. Bar Plot - University Rating Count

In [16]:
Rating_matrix=data['University_Rating'].value_counts().sort_index().reset_index()
Rating_matrix

,University_Rating,count
0,1,34
1,2,126
2,3,162
3,4,105
4,5,73


In [17]:
fig = px.bar(Rating_matrix,y='count', x='University_Rating', title='University Rating Distribution')
fig.update_layout(template='plotly_dark',yaxis_title='Number of applicants')
fig.show(renderer='iframe')



### 📌 **Inference:** Most applicants applied to universities with a rating of 3. It's unusual that university with rating 2 got higher applications than university with rating 4 and 5.

# 📉 5. GRE vs Chance of Admit

In [18]:
fig = px.scatter(data, x='GRE', y='Chance_of_Admit', color='University_Rating',
                 trendline='ols', title='GRE vs Chance of Admit (Colored by University Rating)')
fig.update_layout(template='plotly_dark')
fig.show(renderer='iframe')

### 📌 **Inference:** Higher GRE scores tend to correlate with a higher chance of admission, especially in top-rated universities.

# 📉 6. CGPA vs Chance of Admit

In [19]:
fig = px.scatter(data, x='CGPA', y='Chance_of_Admit', color='Research',
                 trendline='ols', title='CGPA vs Chance of Admit (Colored by Research)')
fig.update_layout(template='plotly_dark')
fig.show(renderer='iframe')

### 📌 **Inference:** Research experience boosts admission chances at similar CGPA levels.

# 📊 7. Box Plot - Research vs Chance of Admit

In [20]:
fig = px.box(data, x="Research", y="Chance_of_Admit", title="Chance of Admit by Research Experience")
fig.update_layout(template='plotly_dark')
fig.show(renderer='iframe')


### 📌 **Inference:** Applicants with research experience (1) have higher median chances of admission.

# 📊 8. Heatmap - Correlation Matrix

In [21]:
corr = data.corr().round(2)

fig = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=corr.columns,
    y=corr.columns,
    colorscale='Cividis',
    text=corr.values,
    texttemplate="%{text}"
))
fig.update_layout(title='Correlation Heatmap',template='plotly_dark')
fig.show(renderer='iframe')

### 📌 **Inference:** CGPA, TOEFL and GRE have the strongest correlation with Chance_of_Admit.

# 📈 9. 3D Scatter Plot - GRE, CGPA, TOEFL vs Chance

In [22]:
fig = px.scatter_3d(data, x='GRE', y='CGPA', z='TOEFL',
                    color='Chance_of_Admit',
                    title="3D Scatter: GRE, CGPA, TOEFL vs Chance of Admit")
fig.show(renderer='iframe')


### 📌 **Inference:** High GRE + CGPA + TOEFL lead to higher admission chances (yellow zone).

# 📊 10. SOP & LOR vs Chance of Admit

In [23]:
fig = px.scatter(data, x='SOP', y='Chance_of_Admit', size='LOR', color='LOR',
                 title='SOP vs Chance of Admit (Bubble Size = LOR Strength)')
fig.show(renderer='iframe')

### 📌 **Inference:** Strong LOR combined with high SOP enhances admission chances.

# **<i>Let's do Prediction now!!</i>**<br>
## We'll Predict chances of Admission.

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [25]:
data.head(5)

,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [26]:
Y=data['Chance_of_Admit']


In [27]:
X=data.iloc[:,[0,1,2,3,4,5,6]]

In [28]:
Y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance_of_Admit, Length: 500, dtype: float64

In [29]:
X

,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332,108,5,4.5,4.0,9.02,1
496,337,117,5,5.0,5.0,9.87,1
497,330,120,5,4.5,5.0,9.56,1
498,312,103,4,4.0,5.0,8.43,0


In [30]:

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [31]:
# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



 
<div style="background-color: #f0f0f0; 
            padding: 20px; 
            font: italic 25px Georgia; 
            color: #333; 
            border-left: 10px solid gray;">
Identify the top algorithms with the highest accuracy and determine the best algorithm.
</div>


In [32]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV


In [33]:

# Define all regression models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'Bagging': BaggingRegressor(),
    'Support Vector Regressor': SVR(),
    'K-Nearest Neighbors': KNeighborsRegressor()
}

# Function to evaluate all regression models
def evaluate_models(X_train, X_test, y_train, y_test):
    results = []
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        results.append((name, r2, rmse))
    
    # Sort by R² score (higher is better)
    results.sort(key=lambda x: x[1], reverse=True)
    return results


In [34]:
results = evaluate_models(X_train_scaled, X_test_scaled, y_train, y_test)

# Print performance via all models

print("Model Performance: \n")
for name, r2, rmse in results:
    print(f"{name}: R² = {r2:.8f}, RMSE = {rmse:.4f}")


Model Performance: 

Linear Regression: R² = 0.81884326, RMSE = 0.0609
Ridge Regression: R² = 0.81878854, RMSE = 0.0609
Random Forest: R² = 0.78908920, RMSE = 0.0657
Gradient Boosting: R² = 0.78247113, RMSE = 0.0667
K-Nearest Neighbors: R² = 0.78020929, RMSE = 0.0670
Bagging: R² = 0.78008117, RMSE = 0.0671
Support Vector Regressor: R² = 0.75893846, RMSE = 0.0702
AdaBoost: R² = 0.75160949, RMSE = 0.0713
Decision Tree: R² = 0.62718826, RMSE = 0.0873
Lasso Regression: R² = -0.00724844, RMSE = 0.1435
ElasticNet: R² = -0.00724844, RMSE = 0.1435


## Linear Regression gives best R^2 score

### Let's tune this more so that R^2 score increases

In [35]:
lr=LinearRegression(fit_intercept=True,copy_X=False,positive=True)
lr.fit(X_train_scaled, y_train)
y_pred1 = lr.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred1)
rmse = mean_squared_error(y_test, y_pred1, squared=False)


In [36]:
r2

0.8188432567829628

In [37]:
rmse

0.06086588041578312

#### Tuning Linear Regression algorithm doesnot effect model performance much

### Let's tune Ridge Regression more so that R^2 score increases

In [38]:
for solver in ['auto','svd', 'cholesky', 'lsqr', 'sag', 'saga']:
    ridge=Ridge(alpha=0.01,solver=solver,max_iter=10000)
    ridge.fit(X_train_scaled, y_train)
    y_pred1 = ridge.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred1)
    rmse = mean_squared_error(y_test, y_pred1, squared=False)
    print(solver,"   r2=",r2,"rmse=",rmse)

auto    r2= 0.81884271273969 rmse= 0.06086597181080196
svd    r2= 0.8188427127396901 rmse= 0.06086597181080194
cholesky    r2= 0.81884271273969 rmse= 0.06086597181080196
lsqr    r2= 0.8188427126260965 rmse= 0.06086597182988477
sag    r2= 0.8188418378944665 rmse= 0.06086611877767597
saga    r2= 0.8188401982645724 rmse= 0.06086639422119149


In [39]:
ridge=Ridge(alpha=0.01,solver='lbfgs',positive=True)
ridge.fit(X_train_scaled, y_train)
y_pred12 = ridge.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred12)
rmse = mean_squared_error(y_test, y_pred12, squared=False)
print(solver,"   r2=",r2,"rmse=",rmse)

saga    r2= 0.818623644927575 rmse= 0.06090276234483202


In [40]:
## Clearly saga solver is yielding higher r2 score.

### Let's tune Random Forest more so that R^2 score increases

In [41]:

# Define model
rf = RandomForestRegressor()

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200,250],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# Setup GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit on training data
grid_search.fit(X_train_scaled, y_train)

# Best model prediction
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_scaled)

# Calculate R² score
r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)

R² Score: 0.795491633770427


### From above we can see that Ridge seems best algorithm from all with saga solver

# Upvote if you found it useful!!